In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import csv
import skvideo.io
from tqdm import tqdm
%matplotlib inline

In [4]:
train = 'train'
test = 'test'

TRAIN_PATH = os.path.join('../',train)
TEST_PATH = os.path.join('../',test)
print(TRAIN_PATH, TEST_PATH)

VID_PATH = 'videos/'
IMG_PATH = 'photos/images'

TRAIN_VID = os.path.join(TRAIN_PATH,VID_PATH+'trainGray.mp4')
TEST_VID = os.path.join(TEST_PATH,VID_PATH+'testGray.mp4')

TRAIN_IMG = os.path.join(TRAIN_PATH,IMG_PATH+'train_imgs')
TEST_IMG = os.path.join(TEST_PATH,IMG_PATH+'test_imgs')

train_frame = 20400
test_fram = 10798

../train ../test
../train/hello


In [ ]:
train_y = list(pd.read_csv(os.path.join(TRAIN_PATH, 'train.txt'), header=None, squeeze=True))
assert(len(train_y)==train_frames)

In [ ]:
def dataset_constructor(video_loc, img_folder, tot_frames, dataset_type):
    meta_dict = {}

    tqdm.write('reading in video file...')
    tot_frames = train_frames
    cap = skvideo.io.vread(video_loc)
     
    tqdm.write('constructing dataset...')
    for idx, frame in enumerate(tqdm(cap)):    
        img_path = os.path.join(img_folder, str(idx)+'.jpg')
        frame_speed = float('NaN') if dataset_type == 'test' else train_y[idx]
        meta_dict[idx] = [img_path, idx, frame_speed]
        skvideo.io.vwrite(img_path, frame)
    meta_df = pd.DataFrame.from_dict(meta_dict, orient='index')
    meta_df.columns = ['image_path', 'image_index', 'speed']
    
    tqdm.write('writing meta to csv')
    meta_df.to_csv(os.path.join(CLEAN_DATA_PATH, dataset_type+'_meta.csv'), index=False)
    
    return "done dataset_constructor"

In [ ]:
# train data
dataset_constructor(TRAIN_VIDEO, CLEAN_IMGS_TRAIN, train_frames, 'train')

In [ ]:
# test data
dataset_constructor(TEST_VIDEO, CLEAN_IMGS_TEST, test_frames, 'test')

In [ ]:
train_meta = pd.read_csv(os.path.join(CLEAN_DATA_PATH, 'train_meta.csv'))
assert(train_meta.shape[0] == train_frames)
assert(train_meta.shape[1] == 3)

In [ ]:
train_meta.head()

In [ ]:
for i in range(50):
    print('speed:',train_meta['speed'][i] )
    img=mpimg.imread(train_meta['image_path'][i])
    print('shape:', img.shape)
    plt.imshow(img)
    plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
plt.plot(train_meta['speed'])
plt.xlabel('image_index (or time since start)')
plt.ylabel('speed')
plt.title('Speed vs time')
plt.show()

In [ ]:
test_meta = pd.read_csv(os.path.join(CLEAN_DATA_PATH, 'test_meta.csv'))
assert(test_meta.shape[0] == test_frames)
assert(test_meta.shape[1] == 3)

In [ ]:
test_meta.head()

In [ ]:
for i in range(50):
    print('speed:',test_meta['speed'][i] )
    img=mpimg.imread(test_meta['image_path'][i])
    print('shape:', img.shape)
    plt.imshow(img)
    plt.show()